In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-04-21 10:42:22--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-04-21 10:42:22--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?TabwC47Mr4YoeOXAN_6DqGPPdm1uo45arQa_oBEqkFvc-CxyvQHZdwSBDR2REKG-FlrhNCXcUOxXz7hpAkIrxfFWqEGruae_AwfRTXeQold7u_o

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-3dmjvn3v
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-3dmjvn3v
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=2761b8588acb58e7526bcd7d42b9392de1b3aaeb70074305dff7bb9c0e628dc3
  Stored in directory: /tmp/pip-ephem-wheel-cache-6wbhznql/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
1 2 3 4 5 6 7 8

3  2 7 4  11 6 15 8

In [ ]:
%%cu
#include<bits/stdc++.h>

using namespace std;

void vector_sum(vector<float> a,float &cpu_res,int n) {
    for (int i = 0; i < n; i++) {
        cpu_res += a[i];
    }
}

void vector_min(vector<float> a,float &cpu_res,int n){
    cpu_res = INT_MAX;
    for (int i = 0; i < n; i++) {
        cpu_res = min(cpu_res,a[i]);
    }
}

void vector_max(vector<float> a,float &cpu_res,int n){
    cpu_res = INT_MIN;
    for (int i = 0; i < n; i++) {
        cpu_res = max(cpu_res,a[i]);
    }
}

void vector_sd(vector<float> a,float sum,double &cpu_res_sd,int n){
    double mean = (double)sum/(double)n;

    double s = 0;
    for(int i=0;i<n;i++){
        s += ((a[i]-mean)*(a[i]-mean));
    }

    cpu_res_sd = (double)s/(double)n;
}

__global__ void cuda_vector_sum(float* a,int n) {
    const int tid=threadIdx.x;
    int no_of_threads=blockDim.x;

    for(int step=1;step < n; step *= 2,no_of_threads /= 2){
      if (tid <= no_of_threads){
        int ind=2*step*tid;
      
        if((ind+step) >= n){
          a[ind] = a[ind] + 0;
        }else{
          a[ind] = a[ind] + a[ind+step];
        }
      }
    }
}

__global__ void cuda_vector_min(float* a,int n) {
    const int tid=threadIdx.x;
    int no_of_threads=blockDim.x;

    for(int step=1;step < n; step *= 2,no_of_threads /= 2){
      if (tid <= no_of_threads){
        int ind=2*step*tid;
      
        if((ind+step) >= n){
          a[ind] = min(a[ind],FLT_MAX);
        }else{
          a[ind] = min(a[ind],a[ind+step]);
        }
      }
    }
}

__global__ void cuda_vector_max(float* a,int n) {
    const int tid=threadIdx.x;
    int no_of_threads=blockDim.x;

    for(int step=1;step < n; step *= 2,no_of_threads /= 2){
      if (tid <= no_of_threads){
        int ind=2*step*tid;
      
        if((ind+step) >= n){
          a[ind] = max(a[ind],FLT_MIN);
        }else{
          a[ind] = max(a[ind],a[ind+step]);
        }
      }
    }
}

__global__ void cuda_update_arr(float *a,double mean){
    const int tid=threadIdx.x;
    a[tid] = (a[tid]-mean)*(a[tid]-mean);
}

int main() {
    int N = 2048;

    vector<float> a(N);
    srand(time(0));
    generate(begin(a), end(a), []() { return (float(rand())/float((RAND_MAX)) * 100.0); });
  
    for(auto item:a)
      cout<<item<<" ";
    cout<<'\n';

    float cpu_res=0,gpu_res=0;
    double cpu_res_sd = 0,gpu_res_sd = 0;

    cout<<"CPU: "<<'\n';
  //-------------------------------------------------------------------
    
    // Sum calculation
    vector_sum(a,cpu_res,N);
    cout << "Vector Sum using CPU :"<<cpu_res<<" \n";

    // Average calculation
    cout << "Vector Average using CPU :"<<(double)cpu_res/(double)N<<" \n";

    vector_sd(a,cpu_res,cpu_res_sd,N);
    cout << "Vector Standard Deviation using CPU :"<<fixed<<setprecision(2)<<sqrt(cpu_res_sd)<<" \n";

    vector_min(a,cpu_res,N);
    cout << "Vector Min using CPU :"<<cpu_res<<" \n";

    vector_max(a,cpu_res,N);
    cout << "Vector Max using CPU :"<<cpu_res<<" \n";
    

  cout<<"GPU: "<<'\n';
  //-------------------------------------------------------------------
    // Allocate memory on the device
    size_t bytes = sizeof(float) * N;
    float* d_a;
    cudaMalloc(&d_a, bytes);
  //-------------------------------------------------------------------

    // Copy data from the host to the device (CPU to GPU)
   
    

    cudaMemcpy(d_a, a.data(), bytes, cudaMemcpyHostToDevice);
    cuda_vector_sum <<<1,N/2>>> (d_a,N);
    cudaMemcpy(&gpu_res, d_a, sizeof(float), cudaMemcpyDeviceToHost);

    cout << "Vector Sum using GPU :"<<gpu_res<<" \n";

//-------------------------------------------------------------------

    cout << "Vector Average using GPU :"<<(double)gpu_res/(double)N<<" \n";

//-------------------------------------------------------------------

    double mean = (double)gpu_res/(double)N;
    cudaMemcpy(d_a, a.data(), bytes, cudaMemcpyHostToDevice);
    cuda_update_arr<<<1,N>>>(d_a,mean);
    cuda_vector_sum<<<1,N/2>>>(d_a,N);
    cudaMemcpy(&gpu_res, d_a, sizeof(float), cudaMemcpyDeviceToHost);
    gpu_res = (double)gpu_res/(double)N;
    cout << "Vector Standard Deviation using GPU :"<<fixed<<setprecision(2)<<sqrt(gpu_res)<<" \n";
    

//-------------------------------------------------------------------

    cudaMemcpy(d_a, a.data(), bytes, cudaMemcpyHostToDevice);
    gpu_res = INT_MAX;
    cuda_vector_min <<<1,N/2>>> (d_a,N);
    cudaMemcpy(&gpu_res, d_a, sizeof(float), cudaMemcpyDeviceToHost);
    cout << "Vector Min using GPU :"<<gpu_res<<" \n";

//-------------------------------------------------------------------

    cudaMemcpy(d_a, a.data(), bytes, cudaMemcpyHostToDevice);
    gpu_res = INT_MIN;
    cuda_vector_max <<<1,N/2>>> (d_a,N);
    cudaMemcpy(&gpu_res, d_a, sizeof(float), cudaMemcpyDeviceToHost);
    cout << "Vector Max using GPU :"<<gpu_res<<" \n";

//-------------------------------------------------------------------

    // Free memory on device
    cudaFree(d_a);
}

19.8308 82.3556 70.6371 91.3423 16.6892 1.80691 74.5038 25.3668 39.7864 15.9772 64.519 14.575 97.5904 85.8449 4.64595 96.5589 0.281131 30.1441 15.5312 93.442 35.457 62.8144 61.6007 23.1637 73.3524 70.7867 16.8818 67.0155 39.1608 72.107 39.2931 58.9916 54.4626 9.93016 50.3339 71.1519 11.7371 24.8377 96.5187 51.5234 40.8149 61.0377 66.0984 38.4053 46.8827 70.7444 34.9642 47.1638 0.888477 50.4955 40.6058 36.3455 13.3099 2.20654 59.5092 86.6623 72.9932 76.391 53.6778 12.154 48.498 92.9709 71.1456 2.96063 2.90106 21.4795 74.1125 14.6381 46.3171 70.6312 66.1616 87.1321 31.669 32.26 25.5374 78.5516 3.00437 60.5016 25.7154 3.89285 10.9971 66.3212 40.2384 24.307 68.5278 99.7476 10.9693 41.521 76.1386 64.6471 53.6749 24.6366 57.618 24.8205 27.5972 60.519 46.3 1.7097 75.1572 92.6171 72.3409 41.3187 79.7492 4.00987 73.5788 5.28658 82.5615 76.5831 65.7882 8.27691 80.476 76.7853 74.5982 20.7143 1.09225 43.1259 20.4619 12.0615 84.6469 96.6005 76.7086 38.3219 21.2371 34.3266 63.1424 48.8343 94.8456 9.